<table class="ee-notebook-buttons" align="left">
    <td><a target="_blank"  href="https://github.com/giswqs/geemap/tree/master/examples/template.ipynb"><img width=32px src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" /> View source on GitHub</a></td>
    <td><a target="_blank"  href="https://nbviewer.jupyter.org/github/giswqs/geemap/blob/master/examples/template.ipynb"><img width=26px src="https://upload.wikimedia.org/wikipedia/commons/thumb/3/38/Jupyter_logo.svg/883px-Jupyter_logo.svg.png" />Notebook Viewer</a></td>
    <td><a target="_blank"  href="https://mybinder.org/v2/gh/giswqs/geemap/master?filepath=examples/template.ipynb"><img width=58px src="https://mybinder.org/static/images/logo_social.png" />Run in binder</a></td>
    <td><a target="_blank"  href="https://colab.research.google.com/github/giswqs/geemap/blob/master/examples/template.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" /> Run in Google Colab</a></td>
</table>

## Install Earth Engine API and geemap
Install the [Earth Engine Python API](https://developers.google.com/earth-engine/python_install) and [geemap](https://github.com/giswqs/geemap). The **geemap** Python package builds on the [ipyleaflet](https://github.com/jupyter-widgets/ipyleaflet) package and implements several methods for displaying Earth Engine data layers, such as `Map.addLayer()`, `Map.setCenter()`, and `Map.centerObject()`.
The following script checks if the geemap package has been installed. If not, it will install geemap, which automatically installs its dependencies, including earthengine-api and ipyleaflet.

In [1]:
import subprocess

try:
    import geemap
except ImportError:
    print('geemap package not installed. Installing ...')
    subprocess.check_call(["python", '-m', 'pip', 'install', 'geemap'])

In [2]:
import ee
import geemap

try:
    ee.Initialize()
except Exception as e:
    ee.Authenticate()
    ee.Initialize()

## Create an interactive map

In [3]:
Map = geemap.Map(center=(40, -100), zoom=4)
Map.add_minimap()
Map

Map(center=[40, -100], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_out_t…

## Add Earth Engine Python script

In [4]:
# Add Earth Engine dataset
image = ee.Image('USGS/SRTMGL1_003')

# Set visualization parameters.
vis_params = {
  'min': 0,
  'max': 4000,
  'palette': ['006633', 'E5FFCC', '662A00', 'D8D8D8', 'F5F5F5']}

# Print the elevation of Mount Everest.
xy = ee.Geometry.Point([86.9250, 27.9881])
elev = image.sample(xy, 30).first().get('elevation').getInfo()
print('Mount Everest elevation (m):', elev)

# Add Earth Eninge layers to Map
Map.addLayer(image, vis_params, 'STRM DEM', True, 0.5)
Map.addLayer(xy, {'color': 'red'}, 'Mount Everest')

Mount Everest elevation (m): 8729


## Change map positions

In [5]:
Map.centerObject(xy, 13)

In [6]:
Map.setCenter(-100, 40, 4)

## Add custom TileLayer

In [7]:
url = 'https://mt1.google.com/vt/lyrs=m&x={x}&y={y}&z={z}'
Map.add_tile_layer(url, name='Google Map', attribution='Google')

## Add custom WMS TileLayer

More WMS layers can be found at <https://viewer.nationalmap.gov/services/>

In [8]:
url = 'https://services.nationalmap.gov/arcgis/services/USGSNAIPImagery/ImageServer/WMSServer?'
Map.add_wms_layer(url=url, layers='0', name='NAIP Imagery', format='image/png')

In [9]:
url = 'https://elevation.nationalmap.gov/arcgis/services/3DEPElevation/ImageServer/WMSServer?'
Map.add_wms_layer(url=url, layers='3DEPElevation:None', name='3DEP Elevation')

## Capture user interaction

In [10]:
import geemap
from ipywidgets import Label
from ipyleaflet import Marker

Map = geemap.Map(center=(40, -100), zoom=4)

label = Label()
display(label)

coordinates = []

def handle_interaction(**kwargs):
    latlon = kwargs.get('coordinates')
    if kwargs.get('type') == 'mousemove':
        label.value = str(latlon)
    elif kwargs.get('type') == 'click':
        coordinates.append(latlon)
        Map.add_layer(Marker(location=latlon))

Map.on_interaction(handle_interaction)

Map

Label(value='')

Map(center=[40, -100], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_out_t…

In [11]:
print(coordinates)

[]


## Extract information from Earth Engine based on user interaction

In [12]:
import geemap
from ipywidgets import Label
from ipyleaflet import Marker

Map = geemap.Map(center=(40, -100), zoom=4)

# Add Earth Engine dataset
image = ee.Image('USGS/SRTMGL1_003')

# Set visualization parameters.
vis_params = {
  'min': 0,
  'max': 4000,
  'palette': ['006633', 'E5FFCC', '662A00', 'D8D8D8', 'F5F5F5']}

# Add Earth Eninge layers to Map
Map.addLayer(image, vis_params, 'STRM DEM', True, 0.5)

latlon_label = Label()
elev_label = Label()
display(latlon_label)
display(elev_label)
coordinates = []

def handle_interaction(**kwargs):
    latlon = kwargs.get('coordinates')
    if kwargs.get('type') == 'mousemove':
        latlon_label.value = "Coordinates: {}".format(str(latlon))
    elif kwargs.get('type') == 'click':
        coordinates.append(latlon)
        Map.add_layer(Marker(location=latlon))
        xy = ee.Geometry.Point(latlon[::-1])
        elev = image.sample(xy, 30).first().get('elevation').getInfo()
        elev_label.value = "Elevation of {}: {} m".format(latlon, elev)
Map.on_interaction(handle_interaction)

Map

Label(value='')

Label(value='')

Map(center=[40, -100], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_out_t…

## A simplifed version for capturing user interaction

In [13]:
import geemap
Map = geemap.Map(center=(40, -100), zoom=4)
cluster = Map.listening(event='click', add_marker=True)
Map

Map(center=[40, -100], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_out_t…

In [14]:
# Get the last mouse clicked coordinates
Map.last_click

[]

In [15]:
# Get all the mouse clicked coordinates
Map.all_clicks

[]

## SplitMap Control

In [16]:
from ipyleaflet import *

m = Map(center=(47.50, -101), zoom=7, scroll_wheel_zoom=True)

right_layer = WMSLayer(
    url = 'https://ndgishub.nd.gov/arcgis/services/Imagery/AerialImage_ND_2017_CIR/ImageServer/WMSServer?',
    layers = 'AerialImage_ND_2017_CIR',
    name = 'AerialImage_ND_2017_CIR',
    format = 'image/png'
)

left_layer = WMSLayer(
    url = 'https://ndgishub.nd.gov/arcgis/services/Imagery/AerialImage_ND_2018_CIR/ImageServer/WMSServer?',
    layers = 'AerialImage_ND_2018_CIR',
    name = 'AerialImage_ND_2018_CIR',
    format = 'image/png'
)

control = SplitMapControl(left_layer=left_layer, right_layer=right_layer)
m.add_control(control)
m.add_control(LayersControl(position='topright'))
m.add_control(FullScreenControl())

m

Map(center=[47.5, -101], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_out…